## Install `simple_salesforce`  module

Remove the comment symbol (#) to run `!pip install simple_salesforce`. Once this module is installed, you can comment it out again. 

In [1]:
#!pip install simple_salesforce

## Import dependencies

In [2]:
import json
from sqlalchemy import create_engine
import pandas as pd
import pymysql
pymysql.install_as_MySQLdb()

## Import configuration variables

In [3]:
# import sys
# sys.path.append('../../../../')
from config import sf_username, sf_password, sf_security_token
from config import remote_db_endpoint, remote_db_port
from config import remote_db_name, remote_db_user, remote_db_pwd

## Set up `simple_salesforce`

Pass in the username, password, and security token here. The security token contains information about your org.

In [4]:
from simple_salesforce import Salesforce
sf = Salesforce(username=sf_username, password=sf_password, security_token=sf_security_token)

## Connect to your MySQL database

In [5]:
engine = create_engine(f"mysql://{remote_db_user}:{remote_db_pwd}@{remote_db_endpoint}:{remote_db_port}/{remote_db_name}")
conn = engine.connect()

## Query Data

In [6]:
course_data = pd.read_sql("SELECT * FROM course", conn)
course_data.head()

,ID_Course,CourseCode,CourseName,CreditHours,BootCampCourse
0,1,BC-DATAVIZ,Data Visualization and Analytics,12,1
1,2,BC-WEBDEV,Full Stack Web Development,12,1
2,3,BC-UIUX,User Interface/User Experience,12,1
3,4,CIS-349,Introduction to Databases,5,0
4,5,CIS-405,Database Programming,5,0


## Transform Data

In [7]:
course_data = course_data.rename(columns={
                                            'ID_Course':'ID_Course__c',
                                            'CourseCode':'CourseCode__c',
                                            'CourseName':'Name',
                                            'CreditHours':'CreditHours__c',
                                            'BootCampCourse':'BootCampCourse__c'
                                         })
course_data.head()

,ID_Course__c,CourseCode__c,Name,CreditHours__c,BootCampCourse__c
0,1,BC-DATAVIZ,Data Visualization and Analytics,12,1
1,2,BC-WEBDEV,Full Stack Web Development,12,1
2,3,BC-UIUX,User Interface/User Experience,12,1
3,4,CIS-349,Introduction to Databases,5,0
4,5,CIS-405,Database Programming,5,0


In [8]:
'''
# Convert date type to be reable
student_data['FullName'] = student_data['FirstName'].str.cat(student_data['LastName'], sep=" ")
student_data['Company'] = 'Dartling University'
student_data = student_data.rename(columns={'FullName':'Name'})


student_load_df = student_data[['LastName','FirstName','Company']]
student_load_df.head()
'''

'\n# Convert date type to be reable\nstudent_data[\'FullName\'] = student_data[\'FirstName\'].str.cat(student_data[\'LastName\'], sep=" ")\nstudent_data[\'Company\'] = \'Dartling University\'\nstudent_data = student_data.rename(columns={\'FullName\':\'Name\'})\n\n\nstudent_load_df = student_data[[\'LastName\',\'FirstName\',\'Company\']]\nstudent_load_df.head()\n'

## Convert DataFrame to a list of dictionaries 

This can help facilitate a bulk insert

In [9]:
course_load = course_data.to_dict(orient='record')
course_load

[{'ID_Course__c': 1,
  'CourseCode__c': 'BC-DATAVIZ',
  'Name': 'Data Visualization and Analytics',
  'CreditHours__c': 12,
  'BootCampCourse__c': 1},
 {'ID_Course__c': 2,
  'CourseCode__c': 'BC-WEBDEV',
  'Name': 'Full Stack Web Development',
  'CreditHours__c': 12,
  'BootCampCourse__c': 1},
 {'ID_Course__c': 3,
  'CourseCode__c': 'BC-UIUX',
  'Name': 'User Interface/User Experience',
  'CreditHours__c': 12,
  'BootCampCourse__c': 1},
 {'ID_Course__c': 4,
  'CourseCode__c': 'CIS-349',
  'Name': 'Introduction to Databases',
  'CreditHours__c': 5,
  'BootCampCourse__c': 0},
 {'ID_Course__c': 5,
  'CourseCode__c': 'CIS-405',
  'Name': 'Database Programming',
  'CreditHours__c': 5,
  'BootCampCourse__c': 0},
 {'ID_Course__c': 6,
  'CourseCode__c': 'CIS-438',
  'Name': 'Database Administration',
  'CreditHours__c': 5,
  'BootCampCourse__c': 0},
 {'ID_Course__c': 7,
  'CourseCode__c': 'CIS-430',
  'Name': 'Business Systems Programming I',
  'CreditHours__c': 5,
  'BootCampCourse__c': 0},
 

## inserting rows individually

Loop through the DataFrame and assemble indiviudal dictionaries

In [10]:
for row in course_load:

    record = {
        'ID_Course__c': row['ID_Course__c'],
        'CourseCode__c': row['CourseCode__c'],
        'Name': row['Name'],
        'CreditHours__c':row['CreditHours__c'],
        'BootCampCourse__c':row['BootCampCourse__c']
    }
    
    try:
        sf.Course__c.create(record)
    except Exception as e:
        print(e)

## inserting rows in bulk

In [11]:
# insert in bulk for 'Course__c' table
'''
try:
    sf.bulk.Course__c.insert(course_load)
except Exception as e:
    print(e)
'''

'\ntry:\n    sf.bulk.Course__c.insert(course_load)\nexcept Exception as e:\n    print(e)\n'